# Resample from 600 to 400 Hz because Source Localization crashes

In [2]:
import os
import mne
from pathlib import Path
import numpy as np
import preprocess

In [3]:
# Paths
data_path = Path("../../Data")
processed_data_path = data_path / 'Processed Data'

In [4]:
sub_ids = [
    '5186',
    '5295',
    '5648',
    '5675',
    '5873',
    '6100',
    '6106',
    '6310',
]

### Load hdf5 files, filter, notch, and export to vhdr

In [5]:
resampled_path = data_path / 'Resampled'
resampled_path.mkdir(parents=True, exist_ok=True)

In [7]:
RESAMPLE_FREQ = 400

In [8]:
for sub_id in sub_ids:
    # Check if sub is done, if so, skip
    if not os.path.exists(resampled_path / f'{sub_id}_preprocessed-raw.fif'):
        print(f'Subject {sub_id} Full Recording')
        # Load raw
        raw = mne.io.read_raw_fif(processed_data_path / f'{sub_id}_preprocessed-raw.fif')
        # Resample
        raw = raw.resample(RESAMPLE_FREQ)
        # Save
        raw.save(resampled_path / f'{sub_id}_preprocessed-raw.fif')
        
    # Eyes open
    if not os.path.exists(resampled_path / f'{sub_id}_eyes_open-raw.fif'):
        print(f'Subject {sub_id} Eyes Open Recording')
        # Load raw
        raw = mne.io.read_raw_fif(processed_data_path / f'{sub_id}_eyes_open-raw.fif')
        # Resample
        raw = raw.resample(RESAMPLE_FREQ)
        # Save
        raw.save(resampled_path / f'{sub_id}_eyes_open-raw.fif')
        
    preprocess.clear_display()

Subject 6310 Full Recording
Opening raw data file ../../Data/Processed Data/6310_preprocessed-raw.fif...
    Range : 600 ... 1040447 =      1.000 ...  1734.078 secs
Ready.
Writing /home/wanglab/Documents/George Kenefati/Lupus EEG Biomarker/Notebooks/Preprocessing/../../Data/Resampled/6310_preprocessed-raw.fif
Closing /home/wanglab/Documents/George Kenefati/Lupus EEG Biomarker/Notebooks/Preprocessing/../../Data/Resampled/6310_preprocessed-raw.fif
[done]
Subject 6310 Eyes Open Recording
Opening raw data file ../../Data/Processed Data/6310_eyes_open-raw.fif...
    Range : 186001 ... 366373 =    310.002 ...   610.622 secs
Ready.
Writing /home/wanglab/Documents/George Kenefati/Lupus EEG Biomarker/Notebooks/Preprocessing/../../Data/Resampled/6310_eyes_open-raw.fif
Closing /home/wanglab/Documents/George Kenefati/Lupus EEG Biomarker/Notebooks/Preprocessing/../../Data/Resampled/6310_eyes_open-raw.fif
[done]


### Read processed data and raw EDF for annotations. Save cropped eyes open portion

In [10]:
missing_eyes_open_markers = []
for sub_id in sub_ids:
    #  Check if sub is done, if so, skip
    if os.path.exists(processed_data_path / f'{sub_id}_eyes_open-raw.fif'):
        print(f'Subject {sub_id} already processed, skipping...')
        continue
    
    print(f'Subject {sub_id}')
    # Processed FIF
    raw = mne.io.read_raw_fif(processed_data_path / f'{sub_id}_preprocessed-raw.fif')
    
    # Raw EDF with annotations
    raw_edf = preprocess.load_file(sub_id, data_path, extension="edf")
    events, event_ids = mne.events_from_annotations(raw_edf)
    
    # If missing eyes open
    if 'KB-Marker-0 (Eyes open) ' not in event_ids:
        missing_eyes_open_markers.append(sub_id)
        print(f"\nMissing eyes open marker for subject {sub_id}")

        # Set event_ids manually
        event_ids = {
                'KB-Marker-0 (Eyes open) ': 3,
                'KB-Marker-9 (END/STOP test) ': 7
                }

        # Set events manually on subject by subject basis
        sfreq = raw.info['sfreq']
        if sub_id == '5295':
            eyes_open_estimated_time = 182
            stop_estimated_time = eyes_open_estimated_time + 300
            events = np.array([
                [eyes_open_estimated_time*sfreq, 0,  3],
                [stop_estimated_time*sfreq, 0,  7],
            ] )
            
            # Crop eyes open and export to vhdr for plotting
            eyes_open = preprocess.crop_resting_EO(raw, sub_id, data_path, processed_data_path,
                                                   events=events, event_ids=event_ids)
            eyes_open.export(data_path / f'Subject {sub_id}' / 'eyes_open.vhdr', overwrite=True)
            print(f"Subject {sub_id} processed with manual eyes open events")
        else:
            continue
    
        # reset
        events, event_ids = None, None

    # Not missing eyes open
    else:
        # Crop eyes open and export to vhdr for plotting
        eyes_open = preprocess.crop_resting_EO(raw, sub_id, data_path, processed_data_path)
        eyes_open.export(data_path / f'Subject {sub_id}' / 'eyes_open.vhdr', overwrite=True)
        print(f"Subject {sub_id} processed")
    
print(missing_eyes_open_markers)

Subject 5186
Opening raw data file ../../Data/Processed Data/5186_preprocessed-raw.fif...


    Range : 600 ... 1150403 =      1.000 ...  1917.338 secs
Ready.
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Lupus EEG Biomarker/Data/Subject 5186/5186 SLE test pinprick2024.04.09_14.29.43.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1150799  =      0.000 ...  1917.998 secs...
Used Annotations descriptions: ['HA-2023.10.03-Trigger 2 (32 mN pinprick) ', 'HA-2023.10.03-Trigger 4 (512 mN pinprick) ', 'KB-Marker-0 (Eyes open) ', 'KB-Marker-1 (Eyes closed) ', 'KB-Marker-2 (256 mN forear...) ', 'KB-Marker-3 (256 mN forear...) ', 'KB-Marker-9 (END/STOP test) ', 'KB-Marker-a (32 mN forearm) ', 'KB-Marker-d (32 mN abdomen) ', 'KB-Marker-f (512 mN abdomen) ', 'KB-Marker-jj (256 mN MCP jo...) ', 'KB-Marker-k (256 mN MCP jo...) ', 'KB-Marker-m (TEST) ', 'KB-Marker-s (512 mN forearm) ']
Extracting EDF parameters from /home/wanglab/Documents/George Kenefati/Lupus EEG Biomarker/Data/Subject 5186/5186 SLE test pinp

### Crop noisy spans

In [13]:
# MAKE SURE TO COMMENT ONCE THIS IS RUN ONCE.
# IF FORGET, CHECK FOR THE NOISY SPAN IN SIGVIEWER
need_crop = {
    '5186': False,
    '5295': False,
    # '5648': (60.5, 61.5),
    '5675': False,
    '5873': False,
    '6100': False,
    '6106': False,
    '6310': False,
}

In [14]:
for sub_id in sub_ids:
    if need_crop[sub_id] is not False:
        print(f'Subject {sub_id}: cropping noisy span {need_crop[sub_id]}')

        # Load whole raw data and eyes open
        raw_eo = mne.io.read_raw_fif(processed_data_path / f'{sub_id}_eyes_open-raw.fif')

        # Crop based on provided times
        raw_eo_new = preprocess.snip_span(raw_eo, need_crop[sub_id][0], need_crop[sub_id][1])
        raw_eo_new.export(data_path / f'Subject {sub_id}' / 'eyes_open.vhdr', overwrite=True)
        print(f"Subject {sub_id} noisy span cropped")

KeyError: '5648'

In [15]:
for sub_id in sub_ids:
    eyes_open = mne.io.read_raw_fif(processed_data_path / f'{sub_id}_eyes_open-raw.fif')
    
    print(f'Subject {sub_id}')
    print(len(eyes_open.times)/sfreq)

Opening raw data file ../../Data/Processed Data/5186_eyes_open-raw.fif...
    Range : 193081 ... 373333 =    321.802 ...   622.222 secs
Ready.
Subject 5186
300.4216666666667
Opening raw data file ../../Data/Processed Data/5295_eyes_open-raw.fif...
    Range : 109800 ... 289800 =    183.000 ...   483.000 secs
Ready.
Subject 5295
300.00166666666667
Opening raw data file ../../Data/Processed Data/5648_eyes_open-raw.fif...
    Range : 137509 ... 318421 =    229.182 ...   530.702 secs
Ready.
Subject 5648
301.52166666666665
Opening raw data file ../../Data/Processed Data/5675_eyes_open-raw.fif...
    Range : 8053 ... 188053 =     13.422 ...   313.422 secs
Ready.
Subject 5675
300.00166666666667
Opening raw data file ../../Data/Processed Data/5873_eyes_open-raw.fif...
    Range : 136393 ... 320185 =    227.322 ...   533.642 secs
Ready.
Subject 5873
306.32166666666666
Opening raw data file ../../Data/Processed Data/6100_eyes_open-raw.fif...
    Range : 131785 ... 313729 =    219.642 ...   522.8

In [16]:
eyes_open.info['ch_names']

['Fp1',
 'Fpz',
 'Fp2',
 'AF7',
 'AF3',
 'AF4',
 'AF8',
 'F7',
 'F5',
 'F3',
 'F1',
 'Fz',
 'F2',
 'F4',
 'F6',
 'F8',
 'FT7',
 'FC5',
 'FC3',
 'FC1',
 'FCz',
 'FC2',
 'FC4',
 'FC6',
 'FT8',
 'T7',
 'C5',
 'C3',
 'C1',
 'Cz',
 'C2',
 'C4',
 'C6',
 'T8',
 'TP7',
 'CP5',
 'CP3',
 'CP1',
 'CPz',
 'CP2',
 'CP4',
 'CP6',
 'TP8',
 'P7',
 'P5',
 'P3',
 'P1',
 'Pz',
 'P2',
 'P4',
 'P6',
 'P8',
 'PO7',
 'PO3',
 'POz',
 'PO4',
 'PO8',
 'O1',
 'Oz',
 'O2',
 'F9',
 'F10',
 'EOG']